# T3: Analize de Dados do SiaSUS

Forma: Trabalho em grupo com apresentação do ambiente e do código para o professor.

Objetivo:
Nesta primeira parte, que deve estar pronta para a aula do dia 03/11/2025, os grupos devem obter, preparar e carregar em um banco de dados relacional os dados do SIASUS (Sistema de Informações Ambulatoriais do SUS), disponibilizados publicamente pelo DataSUS.

Instruções:
1. 1) Obtenção da base de dados
 - Acesse o repositório do DataSUS e faça o download dos arquivos do SIASUS no formato .dbc.
 - Selecione os arquivos referentes a procedimentos ambulatoriais (ex.: PARS e outros complementares necessários para a descrição da tabela PARS).
2. 2) Conversão de formato
 - Converta os arquivos .dbc para .dbf utilizando as ferramentas recomendadas (ex.: TABWIN).
 - Certifique-se de validar a integridade dos arquivos após a conversão.
3) Visualização dos dados
 - Utilize o TABWIN, software oficial do DataSUS, para visualizar os arquivos .dbf.
 - A partir do TABWIN, gere:
  - Scripts SQL de criação de tabelas.
4) Carga dos dados em SGBD:
 - Insira os dados no banco de dados relacional de sua escolha (ex. MySQL, PostgreSQL, IBM DB2, etc).
 - Para arquivos menores, utilize diretamente os scripts de inserção gerados pelo TABWIN.
 - Para arquivos muito grandes (como o PARS, que contém milhões de registros), gere um arquivo CSV a partir do .dbf e utilize um comando de LOAD (carga em lote) no SGBD escolhido.
 - Se optar pelo MySQL (que foi o utilizado em aula):
  - Pesquise e utilize a instrução LOAD DATA na linha de comando do servidor (para evitar problemas de permissões com o cliente Workbench) conforme a documentação oficial: https://dev.mysql.com/doc/refman/8.4/en/load-data.html
Exemplo de comando LOAD DATA no terminal do servidor: LOAD DATA INFILE 'C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/PARS2508.csv' INTO TABLE pars CHARACTER SET latin1 FIELDS TERMINATED BY ',' ENCLOSED BY '"' LINES  TERMINATED BY '\r\n' IGNORE 1 ROWS; 
5) A continuidade do trabalho, na aula do dia 03/11/2025, depende do ambiente do banco de dados funcionando.

## Principais libs que usaremos: 

In [1]:
# Esse projeto foi feito com o objetivo educativo para a matéria de Programção para Ciência dos dados
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt



In [9]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# /kaggle/input/sus-data-csv/CADGERRS.csv

/kaggle/input/TB_SIGTAW.csv
/kaggle/input/tb_municip.csv
/kaggle/input/S_CID.csv
/kaggle/input/PARS2508.csv
/kaggle/input/CBO.csv
/kaggle/input/CADGERRS.csv
/kaggle/input/rl_municip_micibge.csv
/kaggle/input/tb_micibge.csv


In [21]:
cadgerrs_data = pd.read_csv('/kaggle/input/CADGERRS.csv', na_values = 'NaN', encoding = 'latin1', low_memory = False) # Atribuição de espaços vazios.
cadgerrs_data.head(n = 15)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,CNES,CPF_CNPJ,FANTASIA,RAZ_SOCI,RSOC_MAN,LOGRADOU,NUM_END,COMPLEME,BAIRRO,COD_CEP,...,EMAIL,REGSAUDE,MICR_REG,DISTRSAN,DISTRADM,CODUFMUN,EXCLUIDO,DATAINCL,DATAEXCL,NATUREZA
0,7498,65174925004,RAQUEL CRISTINA DANI,CPF 651.749.250/04-RAQUEL CRISTINA DANI,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,430920,NaN,20200121,99991231,NaN
1,7633,80924174072,BEATRIZ VALLE,CPF 809.241.740/72-BEATRIZ VALLE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,430920,NaN,20200121,99991231,NaN
2,13463,88212113061745,SAO JOAO FARMACIAS 29,CNPJ 88.212.113/0617-45-COMERCIO DE MEDICAMENT...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,431690,NaN,20200115,99991231,NaN
3,13498,88212113076009,SAO JOAO FARMACIAS 36,CNPJ 88.212.113/0760-09-COMERCIO DE MEDICAMENT...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4ª,NaN,NaN,NaN,431690,NaN,20200115,99991231,NaN
4,14826,88212113030351,SAO JOAO FARMACIAS 14,CNPJ 88.212.113/0303-51-COMERCIO DE MEDICAMENT...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4ª,NaN,NaN,NaN,431690,NaN,20200115,99991231,NaN
5,15008,88212113037011,SAO JOAO FARMACIAS 22,CNPJ 88.212.113/0370-11-COMERCIO DE MEDICAMENT...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4ª,NaN,NaN,NaN,431690,NaN,20200115,99991231,NaN
6,15067,24772296000127,CLINICA VASCULAR NIENCHESKI,CNPJ 24.772.296/0001-27-CLINICA DE ANGIOLOGIA ...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,431440,NaN,20200116,99991231,NaN
7,15105,960472088,VANESSA ALECSSANDRA BRUNHERI PEREIRA,CPF 009.604.720/88-VANESSA ALECSSANDRA BRUNHER...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,16,NaN,431490,NaN,20200129,99991231,NaN
8,15237,1324767030,RENATA DUTRA DE OLIVEIRA,CPF 013.247.670/30-RENATA DUTRA DE OLIVEIRA,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,16,NaN,431490,NaN,20200129,99991231,NaN
9,16640,35874383000101,CAROLINE CECCONELLO FISIOTERAPEUTA LTDA,CNPJ 35.874.383/0001-01-CAROLINE CECCONELLO FI...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5,NaN,NaN,NaN,430790,NaN,20200210,99991231,NaN
